# Capitulo 9 - Um objeto pythônico

## Representações de objetos

`repr()`: Devolve uma string que representa o objeto como o desenvolvedor quer vê-lo.

`str()`: Devolve uma string que representa o objeto como o usuário quer vê-lo.

## Retorno da classe Vector

In [19]:
# Vector 2d: até agora, todos os métodos são especiais

from array import array
import math

class Vector2d:
  typecode = 'd'

  def __init__(self, x, y):
    self.x = float(x)
    self.y = float(y)
  
  def __iter__(self):
    return (i for i in (self.x, self.y))
  
  def __repr__(self):
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)
  
  def __str__(self):
    return str(tuple(self))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    
    components = (format(c, fmt_spec) for c in coords)
    
    return outer_fmt.format(*components)

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.hypot(self.x, self.y)
  
  def __bool__(self):
    return bool(abs(self))
  
  def angle(self):
    return math.atan2(self.y, self.x)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

In [17]:
# Instâncias de Vector2d têm várias representações

v1 = Vector2d(3, 4)
print(v1.x, v1.y)
x, y = v1
x, y
v1

v1_clone = eval(repr(v1))
v1 == v1_clone
print(v1)
octets = bytes(v1)
octets
abs(v1)
bool(v1), bool(Vector2d(0, 0))

3.0 4.0
(3.0, 4.0)


(True, False)

## `classmethod` versus `staticmethod`

In [26]:
# Comparando os comportamentos de classmethod e de staticmethod

class Demo:
  @classmethod
  def classmeth(*args):
    return args
  
  @staticmethod
  def statmeth(*args):
    return args

Demo.classmeth()
Demo.classmeth('span')

Demo.statmeth()
Demo.statmeth('span')

('span',)

## Apresentações formatadas

`format_spec` é um especificador de formatação que é:
  - o segundo argumento em `format(my_pbj, format_spec)`.
  - o que estiver após os dois-pontos em um campo de substituição delimitado por {} em uma string de formatação usada com `str.format()`.

In [29]:
brl = 1/2.43
brl
format(brl, '0.4f')
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

In [31]:
format(42, 'b')
format(2/3, '.1%')

'66.7%'

In [33]:
from datetime import datetime

now = datetime.now()
format(now, '%H:%M:%S')
"It's now {:%I:%M %p}".format(now)

"It's now 12:33 AM"

In [38]:
v1 = Vector2d(3, 4)
format(v1)
format(v1, '.3f')
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

## Um Vector2d hashable

In [3]:
v1 = Vector2d(3, 4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [16]:
# Vector 2d

from array import array
import math

class Vector2d:
  __slots__ = ('__x', '__y')
  typecode = 'd'

  def __init__(self, x, y):
    self.__x = float(x)
    self.__y = float(y)
  
  @property
  def x(self):
    return self.__x
  
  @property
  def y(self):
    return self.__y

  def __iter__(self):
    return (i for i in (self.x, self.y))
  
  def __repr__(self):
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)
  
  def __str__(self):
    return str(tuple(self))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    
    components = (format(c, fmt_spec) for c in coords)
    
    return outer_fmt.format(*components)

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.hypot(self.x, self.y)
  
  def __bool__(self):
    return bool(abs(self))

  def __hash__(self):
    return hash(self.x) ^ hash(self.y)

  def angle(self):
    return math.atan2(self.y, self.x)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

In [9]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## Atributos privados e "protegidos" em Python

In [15]:
v1 = Vector2d(3,4)
v1.__dict__
v1._Vector2d__x

3.0

## Economizando espaço com o atributo de classe `__slots__`

In [18]:
# time python3 mem_test.py vector2d_v3.py
# time python3 mem_test.py vector2d_v3_slots.py

## Sobrescrita de atributos de classe

In [28]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd
len(dumpd)

v1.typecode = 'f'
dumpf = bytes(v1)
dumpf
len(dumpf)
Vector2d.typecode

'd'

In [30]:
# ShortVector2d é uma subclasse de Vector2d, que apenas sobrescreve o typecode default

class ShortVector2d(Vector2d):
  typecode = 'f'

sv = ShortVector2d(1/11, 1/27)
sv
len(bytes(sv))

9